In [1]:
%%capture
# Install java
!apt install openjdk-21-jdk

In [2]:
%%capture
!wget "https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar" -P "spark/lib-jars/"
!wget "https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.699/aws-java-sdk-bundle-1.12.699.jar" -P "spark/lib-jars/"
!wget "https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk/1.12.699/aws-java-sdk-1.12.699.jar" -P "spark/lib-jars/"
!wget "https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-core/1.12.699/aws-java-sdk-core-1.12.699.jar" -P "spark/lib-jars/"
!wget "https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-s3/1.12.699/aws-java-sdk-s3-1.12.699.jar" -P "spark/lib-jars/"

In [3]:
!java --version

openjdk 21.0.6 2025-01-21
OpenJDK Runtime Environment (build 21.0.6+7-Ubuntu-122.04.1)
OpenJDK 64-Bit Server VM (build 21.0.6+7-Ubuntu-122.04.1, mixed mode, sharing)


In [4]:
try:  # When on Google Colab, clone the repository to download any necessary cache.
    import google.colab
    repo_path = 'cdd'
    !git -C $repo_path pull origin || git clone https://github.com/hygo2025/cdd-projeto-final.git $repo_path
except:
    repo_path = '.'  # Use the local path if not on Google Colab

fatal: cannot change to 'cdd': No such file or directory
Cloning into 'cdd'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 211 (delta 112), reused 155 (delta 65), pack-reused 0 (from 0)
Receiving objects: 100% (211/211), 93.49 KiB | 5.84 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [ ]:
# %%capture
!pip install -r cdd/requirements.txt

In [ ]:
# %%capture
import pkg_resources
!pip install -e $repo_path

In [15]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"

os.environ["SPARK_DRIVER_EXTRACLASSPATH"] = "/content/spark/lib-jars/*"
os.environ["SPARK_EXECUTOR_EXTRACLASSPATH"] = "/content/spark/lib-jars/*"
os.environ["SPARK_SERIALIZER"] = "org.apache.spark.serializer.KryoSerializer"
os.environ["ENV"] = "dev"
os.environ["PYTHONUNBUFFERED"] = "1"
os.environ["SPARK_EXECUTOR_MEMORY"] = "8g"
os.environ["SPARK_DRIVER_MEMORY"] = "8g"
os.environ["SPARK_MEMORY_FRACTION"] = "0.8"


os.environ["DOWNLOAD_FOLDER"] = "/content/datasets"

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
try:  # When on Google Colab, clone the repository to download any necessary cache.
    import google.colab
    repo_path = 'cdd'
    !git -C $repo_path pull origin || git clone https://github.com/hygo2025/cdd-projeto-final.git $repo_path
except:
    repo_path = '.'  # Use the local path if not on Google Colab

Already up to date.


In [10]:
import sys
import os
import pkg_resources

if repo_path not in sys.path:
    sys.path.append(repo_path)

In [11]:
import cdd

In [12]:
import pandas as pd
full_results = pd.DataFrame()

In [13]:
import pandas as pd
from src.sar_model import SarModel
from src.utils.enums import MovieLensDataset, SimilarityType

def evaluate_sar():
    top_k = 10
    validate_size = 0.25
    time_decay_coefficient = 30
    seed = 42
    df_results = pd.DataFrame()


    # Iterando sobre todas as similaridades definidas na enumeração
    for similarity in SimilarityType:
        print(f"\nTestando similaridade: {similarity.value}")

        # Instanciando o modelo com a similaridade atual
        sar_model = SarModel(
            dataset=MovieLensDataset.ML_1M,
            top_k=top_k,
            validate_size=validate_size,
            time_decay_coefficient=time_decay_coefficient,
            similarity_type=similarity,
            seed=seed
        )

        # Avaliando o modelo
        result = sar_model.evaluate()

        result['version'] = f"similarity_{similarity.value}_top_k_{top_k}_validate_size_{validate_size}_time_decay_coefficient_{time_decay_coefficient}"
        result['algorithm'] = 'sar'

        df_results = pd.concat([df_results, result], ignore_index=True)

        print(f"Resultado para similaridade {similarity.value}:")
        print(result)
    return df_results


In [14]:
sar_results = evaluate_sar()


Testando similaridade: cooccurrence
Inicializando SarModel...
Carregando e preparando os dados...
/content/datasets
ML_1M
Dataset ML_1M não encontrado em /content/datasets/ML_1M. Iniciando download e extração...
Iniciando download de: https://files.grouplens.org/datasets/movielens/ml-1m.zip
Progresso: 100.00%
Download concluído.
Descompactando /content/datasets/ml-1m.zip para /content/datasets/ML_1M...
Descompactação concluída.
/content/datasets
ML_1M
Dados carregados: 1000209 linhas.
Dividindo os dados em treino e teste com python_stratified_split...
Conjunto de treino: 750121 linhas, Conjunto de teste: 250088 linhas.
Iniciando o processo de avaliação do modelo SAR...
Iniciando o processo de predição com o modelo SAR...
Iniciando treinamento do modelo SAR...
Treinamento concluído. Salvando o modelo...
Modelo salvo em: ../data/sar/movielens_sar_model_time_decay_coefficient_30_similarity_type_SimilarityType.COCCURRENCE_top_k_10.model
Modelo SAR treinado e salvo com sucesso.
Gerando rec

In [16]:
full_results = pd.concat([full_results, sar_results], ignore_index=True)
full_results.to_csv('01_sar.csv', index=False)

In [17]:
full_results.head(100)

,MAP,nDCG@K,Precision@K,Recall@K,version,algorithm
0,0.173259,0.281954,0.252086,0.082533,similarity_cooccurrence_top_k_10_validate_size...,sar
1,0.219533,0.345610,0.307864,0.114765,similarity_cosine_top_k_10_validate_size_0.25_...,sar
2,0.006920,0.020386,0.025563,0.005893,similarity_inclusion index_top_k_10_validate_s...,sar
3,0.185099,0.309862,0.279222,0.109562,similarity_jaccard_top_k_10_validate_size_0.25...,sar
4,0.000000,0.000000,0.000000,0.000000,similarity_lexicographers mutual information_t...,sar
5,0.000098,0.000360,0.000414,0.000183,similarity_lift_top_k_10_validate_size_0.25_ti...,sar
6,0.008060,0.018790,0.019472,0.003072,similarity_mutual information_top_k_10_validat...,sar


In [18]:
import itertools
import pandas as pd
from src.utils.enums import MovieLensDataset
from src.ncf_model import NcfModel

def evaluate_ncf():
    # Parâmetros a serem testados
    n_factors_options = [4]
    batch_size_options = [256, 512]
    lr_options = [1e-3]
    epochs_options = [10, 15]

    # Parâmetros fixos
    layer_sizes = [16, 8, 4]
    top_k = 10
    test_size = 0.25
    seed = 42
    dataset = MovieLensDataset.ML_1M

    # Cria um DataFrame vazio para armazenar os resultados
    df_results = pd.DataFrame()

    # Itera por todas as combinações de parâmetros
    for n_factors, batch_size, lr, epochs in itertools.product(
        n_factors_options, batch_size_options, lr_options, epochs_options
    ):
        print(f"\nTestando: n_factors={n_factors}, batch_size={batch_size}, lr={lr}, epochs={epochs}")

        # Instancia o modelo NCF com a configuração atual
        model = NcfModel(
            dataset=dataset,
            n_factors=n_factors,
            batch_size=batch_size,
            lr=lr,
            layer_sizes=layer_sizes,
            epochs=epochs,
            top_k=top_k,
            test_size=test_size,
            seed=seed
        )

        # Avalia o modelo
        result = model.evaluate()

        # Adiciona informações de identificação
        result['algorithm'] = 'ncf'
        result['version'] = f"n_factors_{n_factors}_batch_size_{batch_size}_lr_{lr}_epochs_{epochs}"

        df_results = pd.concat([df_results, result], ignore_index=True)

        print(f"Resultado: {result}")

    return df_results

In [21]:
from src.utils.utils import get_torch_device
print(get_torch_device())

2.0.1+cu117
11.7
cuda:0


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="numpy")

ncf_result = evaluate_ncf()


Testando: n_factors=4, batch_size=256, lr=0.001, epochs=10
Inicializando o modelo NcfModel...
Preparando os dados com prepare_data_pandas...
/content/datasets
ML_1M
/content/datasets
ML_1M
Dividindo os dados com python_chrono_split...
Conjunto de treino: 750121 linhas, Conjunto de teste: 250088 linhas
Filtragem dos dados concluída: o conjunto de teste contém apenas usuários e itens presentes no treino.
Iniciando o processo de avaliação...
Iniciando o processo de predição...
Iniciando o treinamento do modelo...
Salvando arquivos temporários para os conjuntos de treino e teste...
Construindo o dataset NCF...
Construindo o modelo NCF...


/usr/local/lib/python3.11/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [ ]:
full_results = pd.concat([full_results, ncf_result], ignore_index=True)
full_results.to_csv('02_sar_ncf.csv', index=False)

In [ ]:
full_results.head(100)

In [ ]:
import itertools
import pandas as pd
from src.utils.enums import MovieLensDataset
from src.bivae_model import BivaeModel

def evaluate_bivae():
    # Parâmetros a serem testados
    latent_dim_options = [50, 100]
    epochs_options = [300, 500]
    lr_options = [0.001]

    # Parâmetros fixos
    top_k = 10
    batch_size = 1024
    encoder_dims = [100]
    act_func = 'tanh'
    likelihood = 'pois'
    test_size = 0.25
    seed = 42
    dataset = MovieLensDataset.ML_1M

    # Cria um DataFrame vazio para armazenar os resultados
    df_results = pd.DataFrame()

    # Itera por todas as combinações de parâmetros
    for latent_dim, epochs, lr in itertools.product(
        latent_dim_options, epochs_options, lr_options
    ):
        print(f"\nTestando: latent_dim={latent_dim}, epochs={epochs}, lr={lr}")

        # Instancia o modelo BivaeModel com a configuração atual
        model = BivaeModel(
            dataset=dataset,
            top_k=top_k,
            batch_size=batch_size,
            latent_dim=latent_dim,
            encoder_dims=encoder_dims,
            act_func=act_func,
            likelihood=likelihood,
            epochs=epochs,
            lr=lr,
            test_size=test_size,
            seed=seed
        )

        # Avalia o modelo
        result = model.evaluate()

        # Adiciona informações de identificação
        result['algorithm'] = 'bivae'
        result['version'] = f"latent_dim_{latent_dim}_epochs_{epochs}_lr_{lr}"

        # Converte o dicionário de resultado em um DataFrame de uma linha e concatena
        df_results = pd.concat([df_results, pd.DataFrame([result])], ignore_index=True)

        print(f"Resultado: {result}")

    return df_results


In [ ]:
bivae_result = evaluate_bivae()

In [ ]:
full_results = pd.concat([full_results, bivae_result], ignore_index=True)
full_results.to_csv('03_sar_ncf_bivae.csv', index=False)

In [ ]:
full_results.head(100)

In [ ]:
import itertools
import pandas as pd
from src.utils.enums import MovieLensDataset
from src.light_gcn_model import LightGcnModel

def evaluate_lightgcn():
    # Parâmetros a serem testados
    n_layers_options = [2, 3, 4]
    lr_options = [0.005, 0.01]
    epochs_options = [1, 5]

    # Parâmetros fixos
    batch_size = 1024
    top_k = 10
    test_size = 0.2
    seed = 42
    dataset = MovieLensDataset.ML_1M

    # Cria um DataFrame vazio para armazenar os resultados
    df_results = pd.DataFrame()

    # Itera por todas as combinações de parâmetros
    for n_layers, lr, epochs in itertools.product(n_layers_options, lr_options, epochs_options):
        print(f"\nTestando: n_layers={n_layers}, lr={lr}, epochs={epochs}")

        # Instancia o modelo LightGcnModel com a configuração atual
        model = LightGcnModel(
            dataset=dataset,
            n_layers=n_layers,
            batch_size=batch_size,
            lr=lr,
            epochs=epochs,
            top_k=top_k,
            test_size=test_size,
            seed=seed
        )

        # Avalia o modelo
        result = model.evaluate()

        # Adiciona informações de identificação
        result['algorithm'] = 'lightgcn'
        result['version'] = f"n_layers_{n_layers}_lr_{lr}_epochs_{epochs}"

        # Converte o dicionário de resultado em um DataFrame de uma linha e concatena
        df_results = pd.concat([df_results, pd.DataFrame([result])], ignore_index=True)

        print(f"Resultado: {result}")

    return df_results


In [ ]:
lightgcn_result = evaluate_lightgcn()

In [ ]:
full_results = pd.concat([full_results, lightgcn_result], ignore_index=True)
full_results.to_csv('04_sar_ncf_bivae_lightgcn.csv', index=False)

In [ ]:
full_results.head(100)

In [ ]:
import itertools
import pandas as pd
from src.utils.enums import MovieLensDataset
from src.fastai_model import FastAiModel

def evaluate_fastai():
    # Parâmetros a serem testados
    n_factors_options = [40, 50]
    epochs_options = [1, 2]

    # Parâmetros fixos
    test_size = 0.25
    top_k = 10
    seed = 42
    dataset = MovieLensDataset.ML_1M

    # Cria um DataFrame vazio para armazenar os resultados
    df_results = pd.DataFrame()

    # Itera por todas as combinações de parâmetros
    for n_factors, epochs in itertools.product(n_factors_options, epochs_options):
        print(f"\nTestando: n_factors={n_factors}, epochs={epochs}")

        # Instancia o modelo FastAiModel com a configuração atual
        model = FastAiModel(
            dataset=dataset,
            n_factors=n_factors,
            test_size=test_size,
            epochs=epochs,
            top_k=top_k,
            seed=seed
        )

        # Avalia o modelo
        result = model.evaluate()

        # Adiciona informações de identificação
        result['algorithm'] = 'fastai'
        result['version'] = f"n_factors_{n_factors}_epochs_{epochs}"

        # Converte o dicionário de resultado em um DataFrame de uma linha e concatena
        df_results = pd.concat([df_results, pd.DataFrame([result])], ignore_index=True)

        print(f"Resultado: {result}")

    return df_results


In [ ]:
fastai_result = evaluate_fastai()

In [ ]:
full_results = pd.concat([full_results, fastai_result], ignore_index=True)
full_results.to_csv('05_sar_ncf_bivae_lightgcn_fastai.csv', index=False)

In [ ]:
full_results.head(100)

In [ ]:
import itertools
import pandas as pd
from src.utils.enums import MovieLensDataset
from src.als_model import SparkAlsModel

def evaluate_als(spark):
    # Definindo os grids de parâmetros
    max_iter_options = [20]
    rank_options = [10, 20, 30, 40]
    reg_param_options = [0.05]
    alpha_options = [0.1]

    # Parâmetros fixos
    validate_size = 0.25
    top_k = 10
    seed = 42
    dataset = MovieLensDataset.ML_1M

    # Cria um DataFrame vazio para armazenar os resultados
    df_results = pd.DataFrame()

    # Itera por todas as combinações de parâmetros
    for max_iter, rank, reg_param, alpha in itertools.product(
            max_iter_options, rank_options, reg_param_options, alpha_options
    ):
        print(f"\nTestando: maxIter={max_iter}, rank={rank}, regParam={reg_param}, alpha={alpha}")

        # Instanciando o modelo ALS com a configuração atual
        als_model = SparkAlsModel(
            spark=spark,
            dataset=dataset,
            max_iter=max_iter,
            rank=rank,
            reg_param=reg_param,
            alpha=alpha,
            validate_size=validate_size,
            top_k=top_k,
            seed=seed
        )


        # Avaliando o modelo
        result = als_model.evaluate()

        # Adiciona as informações desejadas
        result['algorithm'] = 'als'
        result['version'] = f"max_iter_{max_iter}_rank_{rank}_reg_param_{reg_param}_alpha_{alpha}"

        df_results = pd.concat([df_results, result], ignore_index=True)
        print(f"Resultado: {result}")


    return df_results

In [ ]:
from src.utils.spark_session_utils import create_spark_session

spark = create_spark_session("ALS")
als_result = evaluate_als(spark)

In [ ]:
full_results = pd.concat([full_results, als_result], ignore_index=True)
full_results.to_csv('06_sar_ncf_bivae_lightgcn_fastai_als.csv', index=False)